In [32]:
import pymongo
import requests
from pprint import pprint
from tqdm import tqdm, tqdm_notebook
from textacy.preprocess import  preprocess_text
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [33]:
stopwords = requests.get("https://raw.githubusercontent.com/masdevid/ID-Stopwords/master/id.stopwords.02.01.2016.txt").text.split("\n")

In [3]:
categories = ['news', 'bisnis', 'sports', 'entertainment', 'tekno', 'otomotif', 'health']

In [2]:
myClient = pymongo.MongoClient("mongodb://localhost:27017")
myDatabase = myClient.iStorage
myCollection = myDatabase.iData
collection = myDatabase.dataset

In [25]:
# query col Dataset
def getDataset(category=None):
    data = []
    iQuery = collection.find({
        'category' : '{}'.format(category)
    }).limit(5)
    
    for query in iQuery: data.append(query)
        
    return data

In [26]:
temp = getDataset('health')

In [30]:
pprint(temp)

[{'_id': ObjectId('5d688a544246ccaa8427a235'),
  'category': 'health',
  'cleanContent': 'presiden joko widodo alias jokowi mengawali kegiatan 2019 '
                  'berolahraga menyapa warga kebun raya bogor selasa 1 januari '
                  '2019 deputi bidang protokol pers media sekretariat presiden '
                  'bey machmudin keterangannya diterima jakarta libur 2019 '
                  'dimanfaatkan jokowi jogging menyapa warga kebun raya bogor '
                  'tampil santai mengenakan celana training jaket hitam sepatu '
                  'kets jokowi jogging 13 30 wib kediamannya wisma bayurini '
                  'istana kepresidenan bogor jokowi jogging area kebun raya '
                  'bogor jawa barat jalan dilaluinya jokowi menyapa warga '
                  'masyarakat beraktivitas berolahraga berkumpul keluarganya '
                  'kepala negara wargapun berbondong bondong menghampirinya '
                  'bersalaman berswafoto selamat sehat sehat 

In [20]:
# query col iData
def getiData(url=None):
    data = []
    iQuery = myCollection.find(
        {'url' : '{}'.format(url)},
        {
            'title'   : 1,
            'url'     : 1,
            'content' : 1
        }
        
    )
    
    for query in iQuery: data.append(query)
        
    return data

In [21]:
test = getiData(data[0]['url'])

In [72]:
for t in temp:
    print(getiData(t['url']))

[{'_id': ObjectId('5d688a544246ccaa8427a235'), 'title': 'Awal Tahun Baru 2019 Jokowi Jogging, Intip Manfaat Baiknya', 'url': 'https://gaya.tempo.co/read/1160688/awal-tahun-baru-2019-jokowi-jogging-intip-manfaat-baiknya', 'content': 'Presiden Joko Widodo alias Jokowi mengawali kegiatan tahun 2019 dengan berolahraga dan menyapa warga di sekitar Kebun Raya Bogor, Selasa 1 Januari 2019. Deputi Bidang Protokol, Pers, dan Media Sekretariat Presiden, Bey Machmudin dalam keterangannya yang diterima di Jakarta, menyebutkan libur Tahun Baru 2019 dimanfaatkan Jokowi untuk jogging dan menyapa warga di sekitar Kebun Raya Bogor.\nTampil santai mengenakan celana training dan jaket hitam serta sepatu kets, Jokowi mulai jogging sekitar pukul 13.30 WIB. Dari kediamannya di Wisma Bayurini, Istana Kepresidenan Bogor, Jokowi jogging menuju area Kebun Raya Bogor, Jawa Barat. Di sepanjang jalan yang dilaluinya, Jokowi menyapa warga masyarakat yang sedang beraktivitas, baik itu berolahraga atau sekadar berkum

In [64]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [60]:
def cleanContent(content=None):
    # Step 1
    result = preprocess_text(content,
                            fix_unicode=True, 
                            lowercase=True, 
                            no_urls=True,
                            no_emails=True,
                            no_phone_numbers=True,
                            no_numbers=True,
                            no_currency_symbols=True,
                            no_punct=True)
    
    # Step 2
    text_stopword = []
    words = result.split()
    for word in words:
        if word not in stopwords: text_stopword.append(word)
            
    result = ' '.join(text_stopword)
    
    # Step 3    
    # stemming process
    output   = stemmer.stem(result)
    
    return output

In [61]:
test[0]['content']

'Presiden Joko Widodo alias Jokowi mengawali kegiatan tahun 2019 dengan berolahraga dan menyapa warga di sekitar Kebun Raya Bogor, Selasa 1 Januari 2019. Deputi Bidang Protokol, Pers, dan Media Sekretariat Presiden, Bey Machmudin dalam keterangannya yang diterima di Jakarta, menyebutkan libur Tahun Baru 2019 dimanfaatkan Jokowi untuk jogging dan menyapa warga di sekitar Kebun Raya Bogor.\nTampil santai mengenakan celana training dan jaket hitam serta sepatu kets, Jokowi mulai jogging sekitar pukul 13.30 WIB. Dari kediamannya di Wisma Bayurini, Istana Kepresidenan Bogor, Jokowi jogging menuju area Kebun Raya Bogor, Jawa Barat. Di sepanjang jalan yang dilaluinya, Jokowi menyapa warga masyarakat yang sedang beraktivitas, baik itu berolahraga atau sekadar berkumpul dengan keluarganya.\nMelihat Kepala Negara, wargapun berbondong-bondong menghampirinya untuk bersalaman dan berswafoto. "Selamat tahun baru, Pak. Sehat-sehat ya," kata seorang warga."Iya, terima kasih," jawab Jokowi.\nSetelah be

In [62]:
hasil = cleanContent(test[0]['content'])

In [66]:
hasil

'presiden joko widodo alias jokowi awal giat olahraga sapa warga kebun raya bogor selasa januari deputi bidang protokol pers media sekretariat presiden bey machmudin terang terima jakarta libur manfaat jokowi jogging sapa warga kebun raya bogor tampil santai kena celana training jaket hitam sepatu kets jokowi jogging wib diam wisma bayurini istana presiden bogor jokowi jogging area kebun raya bogor jawa barat jalan lalu jokowi sapa warga masyarakat aktivitas olahraga kumpul keluarga kepala negara warga bondong bondong hampir salam berswafoto selamat sehat sehat ya warga iya terima kasih jokowi olahraga interaksi masyarakat jam wib presiden diam wisma bayurini jokowi usia sibuk dunia kerja malas kota halang olah tubuh olahraga giat mahal usia atas olahraga giat senang untung usia lansir laman seniors co au aktif usia hadiah baik presiden jokowi jogging kebun raya bogor selasa januari foto biro pers setpres olahraga tingkat energi sehat kurang gejala kait tua giat otak emosional tubuh ta

In [83]:
def fixContent():
    dataset = getDataset('health')
    for i in tqdm(range(len(dataset))):
        temp = getiData(dataset[i]['url'])
        dataset[i]['cleanContent'] = cleanContent(temp[0]['content'])
        
    return dataset

In [84]:
final = fixContent()

100%|██████████| 5/5 [00:12<00:00,  2.57s/it]


In [73]:
temp

[{'_id': ObjectId('5d688a544246ccaa8427a235'),
  'category': 'health',
  'title': 'Awal Tahun Baru 2019 Jokowi Jogging, Intip Manfaat Baiknya',
  'url': 'https://gaya.tempo.co/read/1160688/awal-tahun-baru-2019-jokowi-jogging-intip-manfaat-baiknya',
  'cleanContent': 'presiden joko widodo alias jokowi mengawali kegiatan 2019 berolahraga menyapa warga kebun raya bogor selasa 1 januari 2019 deputi bidang protokol pers media sekretariat presiden bey machmudin keterangannya diterima jakarta libur 2019 dimanfaatkan jokowi jogging menyapa warga kebun raya bogor tampil santai mengenakan celana training jaket hitam sepatu kets jokowi jogging 13 30 wib kediamannya wisma bayurini istana kepresidenan bogor jokowi jogging area kebun raya bogor jawa barat jalan dilaluinya jokowi menyapa warga masyarakat beraktivitas berolahraga berkumpul keluarganya kepala negara wargapun berbondong bondong menghampirinya bersalaman berswafoto selamat sehat sehat ya warga iya terima kasih jokowi berolahraga berinter